## 加载数据

数据已经预处理过了

In [17]:
import pandas as pd

# 加载数据
data = pd.read_csv('data_hh/result/pre_2023-2025_with_comp_train.csv', dtype={'flt_no': str})

# 查看前几行数据，确保加载成功
# 显示前两行数据以确保正确加载
print(data.shape)
print(data.head(5))
print(data.tail(5))

(1689609, 31)
   flt_no    cap aircraft  legs  leg_no  duration  pax       a       b  \
0  3c6705  110.0      195     1       1      1.30   97  a9f275  24cc4b   
1  6a67d9  110.0      195     1       1      1.25   67  d56796  24cc4b   
2  de569b   93.0      190     3       3      4.75   22  d56796  ec1b13   
3  284078  162.0      320     1       1      1.83   55  e3d141  d618e3   
4  f8765b  167.0      320     1       1      3.98  166  9550bc  2e1ea1   

        c   unit_price  competitor_price  year  month  day  weekday  quarter  \
0  7f9c2b   470.474227        565.375000  2023      1    1        6        1   
1  7f9c2b   454.925373        578.797468  2023      1    1        6        1   
2  dcf90a  1177.818182        200.000000  2023      1    1        6        1   
3  7f9c2b   669.090909        669.090909  2023      1    1        6        1   
4  7f9c2b  1794.783133        229.804511  2023      1    1        6        1   

   is_weekend  is_holiday_season  month_sin  month_cos  hour

In [18]:
# 检查标准化后的统计信息
print("\n标准化后的统计信息：")
print(data['pax'].describe())


标准化后的统计信息：
count    1.689609e+06
mean     1.222227e+02
std      5.222077e+01
min      2.000000e+01
25%      8.300000e+01
50%      1.280000e+02
75%      1.580000e+02
max      3.110000e+02
Name: pax, dtype: float64


## 编码分类变量

### 新增城市标签

In [19]:
import json
# 加载字典
with open('data_hh/result/encoder/city_labels_航班频率加权图标签.json', 'r') as file:
    city_labels_loaded = json.load(file)

print("加载的字典：", city_labels_loaded)

加载的字典： {'000': 1, '13f': 0, '8e3': 0, 'a18': 0, 'eda': 2, '01a': 1, 'b5a': 0, 'ea1': 0, '03f': 2, '1be': 0, '3bc': 0, '578': 0, '5df': 0, '5fb': 0, '760': 0, '792': 0, '7ff': 0, '90a': 0, '974': 0, 'bbb': 0, 'cb5': 0, 'ffe': 0, '059': 2, '022': 0, '077': 0, '0d7': 0, '0dd': 0, '1a3': 0, '1e9': 0, '1ed': 0, '25c': 0, '311': 0, '354': 0, '451': 0, '510': 0, '5d7': 0, '63a': 0, '666': 0, '6e3': 0, '7f1': 0, '812': 0, '8a1': 0, '9d4': 0, '9e9': 0, 'b3a': 0, 'bff': 0, 'c24': 0, 'c3a': 0, 'c4c': 0, 'c98': 0, 'c9f': 0, 'cb7': 0, 'd7c': 0, 'e76': 0, 'e9c': 0, 'efb': 0, '05c': 2, '141': 0, '157': 0, '709': 0, '8ac': 0, '97f': 0, '988': 0, 'a2f': 0, 'b09': 0, 'c14': 0, 'c4b': 0, 'e5b': 0, 'ec2': 0, '074': 1, '4d0': 0, '695': 0, 'c2c': 0, 'fa9': 0, '08c': 1, '08f': 2, '3c5': 0, '5e8': 0, 'a72': 0, 'd54': 0, 'ff1': 0, '0ac': 1, '0c8': 1, '0ca': 1, 'e09': 0, '0d1': 1, '13e': 0, '1ee': 0, '43e': 0, '0db': 2, '919': 0, 'd08': 0, '0ff': 2, '11b': 1, '123': 2, '5de': 0, '8eb': 0, '124': 1, '130': 2, '1

In [20]:
# 使用 map 对 'a', 'b', 'c', 'from', 'to' 列进行标签化，新增对应的标签列
data['a_label'] = data['a'].map(city_labels_loaded)
data['b_label'] = data['b'].map(city_labels_loaded)
data['c_label'] = data['c'].map(city_labels_loaded)
data['from_label'] = data['from'].map(city_labels_loaded)
data['to_label'] = data['to'].map(city_labels_loaded)

### 新增城市二维嵌入

In [21]:
import json

# 加载字典
with open('data_hh/result/encoder/城市嵌入编码_航班频率加权图.json', 'r') as file:
    city_embeddings = json.load(file)

print("加载的字典：", city_embeddings)


加载的字典： {'000': [-1.6952509880065918, -0.8259809613227844], '13f': [0.7161105871200562, -0.43590694665908813], '8e3': [0.5429399609565735, -0.520667314529419], 'a18': [0.20013581216335297, -0.4142817258834839], 'eda': [-0.15405425429344177, -1.1048771142959595], '01a': [-1.7130950689315796, 0.15070012211799622], 'b5a': [0.5836260318756104, -0.7250841856002808], 'ea1': [0.2437203973531723, 0.056170180439949036], '03f': [-1.0081822872161865, -0.733413577079773], '1be': [0.8263573050498962, -0.8512860536575317], '3bc': [1.4952945709228516, -1.2341231107711792], '578': [1.0146775245666504, -0.5335901975631714], '5df': [0.842978298664093, -0.9285436272621155], '5fb': [0.7287545204162598, -0.4136313796043396], '760': [1.2267078161239624, -0.6878513693809509], '792': [0.1306421011686325, -0.7094545364379883], '7ff': [0.9274259805679321, -0.45611515641212463], '90a': [0.06965449452400208, -0.2664845883846283], '974': [1.24435293674469, -1.1002029180526733], 'bbb': [0.4501871168613434, -0.740858

In [22]:
import pandas as pd
import numpy as np

# 将 city_embeddings 转换为 DataFrame
embedding_df = pd.DataFrame.from_dict(city_embeddings, orient='index', columns=['embedding_1', 'embedding_2'])
embedding_df.index.name = 'city'

# 用 'a', 'b', 'c', 'from', 'to' 字段与 embedding_df 合并
data = data.merge(embedding_df, left_on='a', right_index=True, how='left')
data.rename(columns={'embedding_1': 'a_embedding_1', 'embedding_2': 'a_embedding_2'}, inplace=True)

data = data.merge(embedding_df, left_on='b', right_index=True, how='left')
data.rename(columns={'embedding_1': 'b_embedding_1', 'embedding_2': 'b_embedding_2'}, inplace=True)

data = data.merge(embedding_df, left_on='c', right_index=True, how='left')
data.rename(columns={'embedding_1': 'c_embedding_1', 'embedding_2': 'c_embedding_2'}, inplace=True)

data = data.merge(embedding_df, left_on='from', right_index=True, how='left')
data.rename(columns={'embedding_1': 'from_embedding_1', 'embedding_2': 'from_embedding_2'}, inplace=True)

data = data.merge(embedding_df, left_on='to', right_index=True, how='left')
data.rename(columns={'embedding_1': 'to_embedding_1', 'embedding_2': 'to_embedding_2'}, inplace=True)

# 查看添加的新列
print(data[['a_embedding_1', 'a_embedding_2', 'b_embedding_1', 'b_embedding_2', 'c_embedding_1', 'c_embedding_2', 'from_embedding_1', 'from_embedding_2', 'to_embedding_1', 'to_embedding_2']])


         a_embedding_1  a_embedding_2  b_embedding_1  b_embedding_2  \
0                  NaN            NaN            NaN            NaN   
1                  NaN            NaN            NaN            NaN   
2                  NaN            NaN            NaN            NaN   
3                  NaN            NaN            NaN            NaN   
4                  NaN            NaN            NaN            NaN   
...                ...            ...            ...            ...   
1689604            NaN            NaN            NaN            NaN   
1689605            NaN            NaN            NaN            NaN   
1689606            NaN            NaN            NaN            NaN   
1689607            NaN            NaN            NaN            NaN   
1689608            NaN            NaN            NaN            NaN   

         c_embedding_1  c_embedding_2  from_embedding_1  from_embedding_2  \
0                  NaN            NaN         -1.641072          1.691

### 频率编码

使用 json 保存和加载 city_map

In [23]:
# 加载city_map
with open('data_hh/result/encoder/city_map_频率编码.json', 'r') as f:
    city_map = json.load(f)

# 使用 city_map 替换指定列的值
columns_to_replace = ['a', 'b', 'c', 'from', 'to']

# 遍历指定列并直接用 map 映射
for col in columns_to_replace:
    data[col] = data[col].map(city_map)


print(data)

         flt_no    cap aircraft  legs  leg_no  duration  pax   a   b   c  \
0        3c6705  110.0      195     1       1      1.30   97 NaN NaN NaN   
1        6a67d9  110.0      195     1       1      1.25   67 NaN NaN NaN   
2        de569b   93.0      190     3       3      4.75   22 NaN NaN NaN   
3        284078  162.0      320     1       1      1.83   55 NaN NaN NaN   
4        f8765b  167.0      320     1       1      3.98  166 NaN NaN NaN   
...         ...    ...      ...   ...     ...       ...  ...  ..  ..  ..   
1689604  2f5ac7  161.0      738     3       2      2.58  128 NaN NaN NaN   
1689605  8b52d8  161.0      738     1       1      2.60  152 NaN NaN NaN   
1689606  cac556  161.0      738     1       1      1.77  149 NaN NaN NaN   
1689607  0bd968  188.0      7MZ     1       1      2.40  144 NaN NaN NaN   
1689608  2befc2  161.0      738     1       1      2.48  142 NaN NaN NaN   

          unit_price  competitor_price  year  month  day  weekday  quarter  \
0        

### 'flt_no', 'bd_type', 'aircraft'编码

In [24]:
import joblib
from sklearn.preprocessing import LabelEncoder
import os

# 定义需要编码的分类特征
# categorical_columns = ['flt_no', 'bd_type', 'aircraft']
categorical_columns = ['flt_no', 'aircraft']

# 从保存的文件中加载编码器并应用到data
for col in categorical_columns:
    # 加载编码器
    encoder_path = os.path.join('data_hh/result/encoder/', f"{col}_encoder_all.pkl")
    le = joblib.load(encoder_path)
    
    try:
        # 对data进行转换
        data[col] = le.transform(data[col])
        print(f"{col}列编码完成")
    except ValueError as e:
        # 如果遇到新的类别，打印错误信息
        print(f"{col}列编码出错: {str(e)}")
        # 找出新的类别
        new_categories = set(data[col]) - set(le.classes_)
        print(f"{col}列中的新类别: {new_categories}")

# 查看编码后的结果
print("\n编码后的前几行数据：")
print(data[categorical_columns].head())

flt_no列编码完成
aircraft列编码完成

编码后的前几行数据：
   flt_no  aircraft
0    1104         2
1    1872         2
2    3916         0
3     721         9
4    4356         9


## 特征和目标分离
我们要预测的是pax字段，其他字段作为特征。

In [25]:
# 特征列
# X = data[['flt_no', 'bd_type', 'cap', 'aircraft',  'leg_no', 'duration', 'a', 'b', 'c', 'year', 'month', 'day', 'weekday','holiday', 'hour', 'minute', 'second', 'from', 'to','unit_price']]
# X = data[['flt_no', 'bd_type', 'cap', 'aircraft', 'legs', 'leg_no', 'duration', 'a', 'b', 'c', 'year', 'month', 'day', 'weekday','hour', 'minute', 'second', 'from', 'to','unit_price']]
# X = data[['flt_no', 'bd_type', 'cap', 'aircraft', 'legs', 'leg_no', 'duration', 'a', 'b', 'c', 'year', 'month', 'day', 'weekday','hour', 'minute', 'second', 'from', 'to','unit_price','a_label' ,'b_label' ,'c_label' ,'from_label' ,'to_label']]

# 有abc，有标签，有嵌入
# X = data[['flt_no', 'bd_type', 'cap', 'aircraft', 'legs', 'leg_no', 'duration', 'a', 'b', 'c', 'year', 'month', 'day', 'weekday','hour', 'minute', 'second', 'from', 'to','unit_price','a_label' ,'b_label' ,'c_label' ,'from_label' ,'to_label','a_embedding_1' , 'a_embedding_2' , 'b_embedding_1','b_embedding_2' , 'c_embedding_1' , 'c_embedding_2' , 'from_embedding_1','from_embedding_2' , 'to_embedding_1' , 'to_embedding_2']]
# X = data[['flt_no', 'cap', 'aircraft', 'legs', 'leg_no', 'duration', 'a', 'b', 'c', 'year', 'month', 'day', 'weekday','hour', 'minute', 'from', 'to','unit_price','a_label' ,'b_label' ,'c_label' ,'from_label' ,'to_label','a_embedding_1' , 'a_embedding_2' , 'b_embedding_1','b_embedding_2' , 'c_embedding_1' , 'c_embedding_2' , 'from_embedding_1','from_embedding_2' , 'to_embedding_1' , 'to_embedding_2']]
import numpy as np
# 存在性检查→缺则补（只在缺失时计算）
_need = ['quarter','is_weekend','is_holiday_season','hour_sin','hour_cos','month_sin','month_cos']
_missing = [c for c in _need if c not in data.columns]
if _missing:
    assert all(col in data.columns for col in ['month','weekday','hour']), '缺少 month/weekday/hour 无法补时间特征'
    data['quarter'] = data['month'].apply(lambda x: (x - 1) // 3 + 1)
    data['is_weekend'] = data['weekday'].isin([5, 6]).astype(int)
    data['is_holiday_season'] = data['month'].isin([1, 2, 7, 8, 10]).astype(int)
    data['hour_sin'] = np.sin(2 * np.pi * data['hour'] / 24)
    data['hour_cos'] = np.cos(2 * np.pi * data['hour'] / 24)
    data['month_sin'] = np.sin(2 * np.pi * data['month'] / 12)
    data['month_cos'] = np.cos(2 * np.pi * data['month'] / 12)

# 新增：竞品价格差异比例特征（相对当前航班）
if 'competitor_price' in data.columns and 'unit_price' in data.columns:
    _diff = data['competitor_price'] - data['unit_price']
    data['comp_price_diff_abs'] = _diff.abs()
    _den = data['unit_price'].replace(0, float('nan')).astype(float)
    data['comp_price_ratio'] = (_diff / _den).replace([np.inf, -np.inf], np.nan).fillna(0.0)
    data['comp_price_ratio_abs'] = data['comp_price_ratio'].abs()

# 确保月度 from/to 均值与 seg_avg_pax 存在（缺了才计算，使用当前 data）
need_cols = {'from_avg_pax_month','to_avg_pax_month','seg_avg_pax'}
if not need_cols.issubset(data.columns):
    from_stats_m = (data.groupby(['from','month'])['pax']
                    .mean().rename('from_avg_pax_month').reset_index())
    to_stats_m   = (data.groupby(['to','month'])['pax']
                    .mean().rename('to_avg_pax_month').reset_index())
    from_all = (data.groupby('from')['pax']
                .mean().rename('from_avg_pax_all').reset_index())
    to_all   = (data.groupby('to')['pax']
                .mean().rename('to_avg_pax_all').reset_index())
    g = float(data['pax'].mean())
    data = (data
            .merge(from_stats_m, on=['from','month'], how='left')
            .merge(to_stats_m,   on=['to','month'],   how='left')
            .merge(from_all,     on='from',           how='left')
            .merge(to_all,       on='to',             how='left'))
    data['from_avg_pax_month'] = data['from_avg_pax_month'].fillna(data['from_avg_pax_all']).fillna(g)
    data['to_avg_pax_month']   = data['to_avg_pax_month'].fillna(data['to_avg_pax_all']).fillna(g)
    data['seg_avg_pax']        = (data['from_avg_pax_month'] + data['to_avg_pax_month']) / 2.0
X = data[['flt_no', 'cap', 'aircraft', 'legs', 'leg_no', 'duration', 'a', 'b', 'c', 'year', 'month', 'day', 'weekday', 'hour', 'minute', 'quarter', 'is_weekend', 'is_holiday_season', 'hour_sin', 'hour_cos', 'month_sin', 'month_cos', 'from', 'to', 'unit_price', 'competitor_price', 'comp_price_diff_abs', 'comp_price_ratio', 'from_avg_pax_month', 'to_avg_pax_month', 'seg_avg_pax', 'a_label', 'b_label', 'c_label', 'from_label', 'to_label', 'a_embedding_1', 'a_embedding_2', 'b_embedding_1', 'b_embedding_2', 'c_embedding_1', 'c_embedding_2', 'from_embedding_1', 'from_embedding_2', 'to_embedding_1', 'to_embedding_2']]

# 删除了abc，但有标签，有嵌入
# X = data[['flt_no', 'bd_type', 'cap', 'aircraft', 'legs', 'leg_no', 'duration', 'year', 'month', 'day', 'weekday','hour', 'minute', 'second', 'unit_price','a_label' ,'b_label' ,'c_label' ,'from_label' ,'to_label','a_embedding_1' , 'a_embedding_2' , 'b_embedding_1','b_embedding_2' , 'c_embedding_1' , 'c_embedding_2' , 'from_embedding_1','from_embedding_2' , 'to_embedding_1' , 'to_embedding_2']]

# 目标列
y = data['pax']

### 对x进行标准化

In [26]:
from sklearn.preprocessing import StandardScaler

# 对所有特征进行标准化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 将标准化后的数据转回 DataFrame 格式
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

# 查看结果
pd.set_option('display.max_columns', None)  # 显示所有列
print(X_scaled.head(1))


# 保存 scaler
joblib.dump(scaler, 'data_hh/result/encoder/standard_scaler_x.pkl')
print("x的标准化器已保存为 standard_scaler_x.pkl")

/data/zy/anaconda3/envs/machine/lib/python3.10/site-packages/sklearn/utils/extmath.py:1137: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/data/zy/anaconda3/envs/machine/lib/python3.10/site-packages/sklearn/utils/extmath.py:1142: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/data/zy/anaconda3/envs/machine/lib/python3.10/site-packages/sklearn/utils/extmath.py:1162: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


     flt_no       cap  aircraft     legs    leg_no  duration   a   b   c  \
0 -0.848829 -1.494606 -1.378712 -0.85182 -0.558079 -1.236975 NaN NaN NaN   

       year     month      day   weekday      hour    minute   quarter  \
0 -0.977504 -1.307816 -1.68186  1.482253 -0.001571  0.479603 -1.051653   

   is_weekend  is_holiday_season  hour_sin  hour_cos  month_sin  month_cos  \
0    1.564266           1.333616 -0.481549 -0.919969   0.368371   1.241001   

       from        to  unit_price  competitor_price  comp_price_diff_abs  \
0 -1.350272  0.203254   -0.567115         -0.203918             -0.29615   

   comp_price_ratio  from_avg_pax_month  to_avg_pax_month  seg_avg_pax  \
0          0.371132           -1.026896         -0.005433     -0.51278   

   a_label  b_label  c_label  from_label  to_label  a_embedding_1  \
0      NaN      NaN      NaN   -3.677972  -0.20304            NaN   

   a_embedding_2  b_embedding_1  b_embedding_2  c_embedding_1  c_embedding_2  \
0            NaN    

### 对y进行标准化

In [27]:
# 对目标列 y 进行标准化
scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))  # 将 y 转换为 2D 数组进行标准化

# 转换回 DataFrame 格式
y_scaled = pd.DataFrame(y_scaled, columns=['pax_scaled'])

# 查看标准化后的 y
print(y_scaled.head())

# 保存 y 的 scaler
joblib.dump(scaler_y, 'data_hh/result/encoder/standard_scaler_y.pkl')
print("y的标准化器已保存为 standard_scaler_y.pkl")

   pax_scaled
0   -0.483002
1   -1.057486
2   -1.919213
3   -1.287280
4    0.838312
y的标准化器已保存为 standard_scaler_y.pkl


## 训练XGBoost模型

In [12]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

# 自定义 SMAPE 函数
def smape(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    diff = np.abs(y_true - y_pred)
    return np.mean(diff / denominator) * 100

# 自定义评估函数
def smape_eval(y_pred, dtrain):
    y_true = dtrain.get_label()
    smape_value = smape(y_true, y_pred)
    return 'SMAPE', smape_value

# 数据划分
X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# 输出数据集大小
print(f'训练集大小: {X_train.shape[0]}')
print(f'验证集大小: {X_val.shape[0]}')
print(f'测试集大小: {X_test.shape[0]}')

# 转换为 DMatrix 格式
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)
dtest = xgb.DMatrix(X_test, label=y_test)

# 评估集（以验证集为优化目标）
evals = [(dtrain, 'train'), (dval, 'validation')]

# 训练与预测逻辑已迁移至下方 MODE 分支单元
print('已设置 evals，训练请在 MODE 分支单元执行。')


训练集大小: 1351687
验证集大小: 168961
测试集大小: 168961
|   iter    |  target   | learni... | max_depth | subsample | colsam... |   alpha   |  lambda_  | num_bo... |
-------------------------------------------------------------------------------------------------------------


/data/zy/anaconda3/envs/machine/lib/python3.10/site-packages/xgboost/training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


[0]	train-rmse:0.91010	train-SMAPE:162.94341	validation-rmse:0.90808	validation-SMAPE:163.05432
[10]	train-rmse:0.48915	train-SMAPE:78.46379	validation-rmse:0.49383	validation-SMAPE:79.35448
[20]	train-rmse:0.40903	train-SMAPE:67.00732	validation-rmse:0.41928	validation-SMAPE:68.31374
[30]	train-rmse:0.38327	train-SMAPE:63.34489	validation-rmse:0.39812	validation-SMAPE:65.00686
[40]	train-rmse:0.36959	train-SMAPE:61.40808	validation-rmse:0.38813	validation-SMAPE:63.43777
[50]	train-rmse:0.36076	train-SMAPE:60.22864	validation-rmse:0.38229	validation-SMAPE:62.52377
[60]	train-rmse:0.35350	train-SMAPE:59.29375	validation-rmse:0.37775	validation-SMAPE:61.87720
[70]	train-rmse:0.34714	train-SMAPE:58.45836	validation-rmse:0.37376	validation-SMAPE:61.28436
[80]	train-rmse:0.34180	train-SMAPE:57.74328	validation-rmse:0.37075	validation-SMAPE:60.83315
[90]	train-rmse:0.33649	train-SMAPE:57.05924	validation-rmse:0.36799	validation-SMAPE:60.43935
[100]	train-rmse:0.33233	train-SMAPE:56.51371	val

[120]	train-rmse:0.46982	train-SMAPE:72.75700	validation-rmse:0.47016	validation-SMAPE:73.05620
[130]	train-rmse:0.46790	train-SMAPE:72.52200	validation-rmse:0.46828	validation-SMAPE:72.82267
[140]	train-rmse:0.46633	train-SMAPE:72.29777	validation-rmse:0.46675	validation-SMAPE:72.60052
[150]	train-rmse:0.46505	train-SMAPE:72.12849	validation-rmse:0.46551	validation-SMAPE:72.43077
[160]	train-rmse:0.46359	train-SMAPE:71.99588	validation-rmse:0.46408	validation-SMAPE:72.30299
[170]	train-rmse:0.46255	train-SMAPE:71.89433	validation-rmse:0.46307	validation-SMAPE:72.20853
[180]	train-rmse:0.46151	train-SMAPE:71.71992	validation-rmse:0.46204	validation-SMAPE:72.03884
[190]	train-rmse:0.46063	train-SMAPE:71.61227	validation-rmse:0.46117	validation-SMAPE:71.93473
[200]	train-rmse:0.45960	train-SMAPE:71.51354	validation-rmse:0.46018	validation-SMAPE:71.84319
[210]	train-rmse:0.45871	train-SMAPE:71.42471	validation-rmse:0.45931	validation-SMAPE:71.75204
[220]	train-rmse:0.45796	train-SMAPE:71.

[130]	train-rmse:0.45902	train-SMAPE:71.58504	validation-rmse:0.45984	validation-SMAPE:71.95133
[140]	train-rmse:0.45701	train-SMAPE:71.32137	validation-rmse:0.45788	validation-SMAPE:71.69954
[150]	train-rmse:0.45549	train-SMAPE:71.11139	validation-rmse:0.45641	validation-SMAPE:71.50133
[160]	train-rmse:0.45390	train-SMAPE:70.91314	validation-rmse:0.45485	validation-SMAPE:71.30445
[170]	train-rmse:0.45247	train-SMAPE:70.74431	validation-rmse:0.45347	validation-SMAPE:71.15209
[180]	train-rmse:0.45117	train-SMAPE:70.56763	validation-rmse:0.45222	validation-SMAPE:70.98843
[190]	train-rmse:0.45007	train-SMAPE:70.45687	validation-rmse:0.45116	validation-SMAPE:70.88290
[200]	train-rmse:0.44911	train-SMAPE:70.33508	validation-rmse:0.45022	validation-SMAPE:70.76379
[210]	train-rmse:0.44801	train-SMAPE:70.20189	validation-rmse:0.44918	validation-SMAPE:70.62869
[220]	train-rmse:0.44684	train-SMAPE:70.02457	validation-rmse:0.44807	validation-SMAPE:70.44814
[230]	train-rmse:0.44587	train-SMAPE:69.

...

/data/zy/anaconda3/envs/machine/lib/python3.10/site-packages/xgboost/training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


[0]	train-rmse:0.87105	train-SMAPE:135.11295	validation-rmse:0.86880	validation-SMAPE:135.24016
[10]	train-rmse:0.54707	train-SMAPE:81.16786	validation-rmse:0.54613	validation-SMAPE:81.31662
[20]	train-rmse:0.51634	train-SMAPE:77.33570	validation-rmse:0.51582	validation-SMAPE:77.54382
[30]	train-rmse:0.50609	train-SMAPE:76.20731	validation-rmse:0.50584	validation-SMAPE:76.44981
[40]	train-rmse:0.49832	train-SMAPE:75.36147	validation-rmse:0.49818	validation-SMAPE:75.61876
[50]	train-rmse:0.49297	train-SMAPE:74.76770	validation-rmse:0.49289	validation-SMAPE:75.02724
[60]	train-rmse:0.48924	train-SMAPE:74.10069	validation-rmse:0.48924	validation-SMAPE:74.34346
[70]	train-rmse:0.48537	train-SMAPE:73.80896	validation-rmse:0.48541	validation-SMAPE:74.04305
[80]	train-rmse:0.48354	train-SMAPE:73.55455	validation-rmse:0.48360	validation-SMAPE:73.81037
[90]	train-rmse:0.48177	train-SMAPE:73.32193	validation-rmse:0.48188	validation-SMAPE:73.58127
[100]	train-rmse:0.48000	train-SMAPE:73.19482	val

/data/zy/anaconda3/envs/machine/lib/python3.10/site-packages/xgboost/training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


[0]	train-rmse:0.89414	train-SMAPE:158.54132	validation-rmse:0.89223	validation-SMAPE:158.65868
[10]	train-rmse:0.46423	train-SMAPE:74.80370	validation-rmse:0.46927	validation-SMAPE:75.73622
[20]	train-rmse:0.40065	train-SMAPE:65.48978	validation-rmse:0.41170	validation-SMAPE:66.80798
[30]	train-rmse:0.37680	train-SMAPE:62.07547	validation-rmse:0.39321	validation-SMAPE:63.93119
[40]	train-rmse:0.36457	train-SMAPE:60.45103	validation-rmse:0.38482	validation-SMAPE:62.72417
[50]	train-rmse:0.35757	train-SMAPE:59.51992	validation-rmse:0.38049	validation-SMAPE:62.07197
[60]	train-rmse:0.34991	train-SMAPE:58.50658	validation-rmse:0.37579	validation-SMAPE:61.38211
[70]	train-rmse:0.34280	train-SMAPE:57.58715	validation-rmse:0.37143	validation-SMAPE:60.76738
[80]	train-rmse:0.33654	train-SMAPE:56.76965	validation-rmse:0.36793	validation-SMAPE:60.24028
[90]	train-rmse:0.33248	train-SMAPE:56.24777	validation-rmse:0.36582	validation-SMAPE:59.92695
[100]	train-rmse:0.32789	train-SMAPE:55.65770	val

[50]	train-rmse:0.35757	train-SMAPE:59.51992	validation-rmse:0.38049	validation-SMAPE:62.07197
[60]	train-rmse:0.34991	train-SMAPE:58.50658	validation-rmse:0.37579	validation-SMAPE:61.38211
[70]	train-rmse:0.34280	train-SMAPE:57.58715	validation-rmse:0.37143	validation-SMAPE:60.76738
[80]	train-rmse:0.33654	train-SMAPE:56.76965	validation-rmse:0.36793	validation-SMAPE:60.24028
[90]	train-rmse:0.33248	train-SMAPE:56.24777	validation-rmse:0.36582	validation-SMAPE:59.92695
[100]	train-rmse:0.32789	train-SMAPE:55.65770	validation-rmse:0.36353	validation-SMAPE:59.60034
[110]	train-rmse:0.32306	train-SMAPE:54.99430	validation-rmse:0.36136	validation-SMAPE:59.27383
[120]	train-rmse:0.31854	train-SMAPE:54.42403	validation-rmse:0.35936	validation-SMAPE:58.99979
[130]	train-rmse:0.31423	train-SMAPE:53.87021	validation-rmse:0.35753	validation-SMAPE:58.74194
[140]	train-rmse:0.31041	train-SMAPE:53.35580	validation-rmse:0.35602	validation-SMAPE:58.50689
[150]	train-rmse:0.30709	train-SMAPE:52.91683

In [28]:
# 运行模式配置（显式选择）
MODE = 'manual'  # 选 'bayes' 或 'manual'

# 手动模式参数来源
USE_SAVED_BEST = True  # manual 模式下，是否从 JSON 读取
BEST_JSON_PATH = 'data_hh/result/xgb_bayes_best_params.json'

# 固定/公共 XGBoost 参数
try:
    BASE_PARAMS
except NameError:
    BASE_PARAMS = {'objective': 'reg:squarederror'}

# 若不读取 JSON，就在这里手动指定参数（示例字段）
MANUAL_PARAMS = {
    # 'learning_rate': 0.08,
    # 'max_depth': 6,
    # 'subsample': 0.8,
    # 'colsample_bytree': 0.8,
    # 'alpha': 0.0,
    # 'lambda': 1.0,
    # 'num_boost_round': 600,
}


In [29]:
import os, json
import numpy as np
import pandas as pd
import xgboost as xgb

# 确保结果目录存在
os.makedirs('data_hh/result/model', exist_ok=True)

def load_best_params(path):
    with open(path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    params = data.get('best_params', data)
    p = dict(params)
    if 'lambda_' in p and 'lambda' not in p:
        p['lambda'] = float(p.pop('lambda_'))
    if 'max_depth' in p: p['max_depth'] = int(p['max_depth'])
    if 'num_boost_round' in p: p['num_boost_round'] = int(p['num_boost_round'])
    for k in ('learning_rate','subsample','colsample_bytree','alpha','lambda'):
        if k in p: p[k] = float(p[k])
    return p

def smape_np(y_true, y_pred):
    y_true = np.asarray(y_true).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)
    denom = np.abs(y_true) + np.abs(y_pred)
    denom = np.where(denom == 0, 1.0, denom)
    return float(100.0 * np.mean(2.0 * np.abs(y_pred - y_true) / denom))

# 评估集（以验证集为优化目标）
try:
    evals
except NameError:
    evals = [(dtrain, 'train'), (dval, 'validation')]

if MODE == 'bayes':
    # 仅在 Bayes 模式下导入调参工具，避免 manual 模式对依赖的要求
    from xgb_bayes_opt import bayesian_tune_xgb
    init_probe = None
    if os.path.exists(BEST_JSON_PATH):
        try:
            bp = load_best_params(BEST_JSON_PATH)
            init_probe = {
                'learning_rate': bp.get('learning_rate'),
                'max_depth': bp.get('max_depth'),
                'subsample': bp.get('subsample'),
                'colsample_bytree': bp.get('colsample_bytree'),
                'alpha': bp.get('alpha'),
                'lambda_': bp.get('lambda'),
                'num_boost_round': bp.get('num_boost_round'),
            }
        except Exception as e:
            print(f'加载 BEST_JSON_PATH 失败，忽略 warm-start: {e}')
    model, best_params, best_smape = bayesian_tune_xgb(
        dtrain,
        evals,
        base_params=BASE_PARAMS,
        init_points=8,
        n_iter=20,
        early_stopping_rounds=10,
        verbose_eval=10,
        save_model_name='xgb_model_bayes.json',
        init_probe_params=init_probe,
    )
    print('Bayes Best SMAPE:', best_smape)
    print('Bayes Best Params:', best_params)
    y_pred = model.predict(dtest)
    try:
        y_true = y_test.values if hasattr(y_test, 'values') else y_test
    except NameError:
        y_true = None
    df_out = pd.DataFrame({'y_pred': y_pred})
    if y_true is not None:
        df_out['y_true'] = np.asarray(y_true).reshape(-1)
        print('Test SMAPE:', smape_np(df_out['y_true'], df_out['y_pred']))
    # 反标准化并计算原始尺度 SMAPE（若提供 scaler_y）
    try:
        scaler_y
        has_scaler = True
    except NameError:
        has_scaler = False
    if has_scaler and hasattr(scaler_y, 'inverse_transform'):
        y_pred_orig = scaler_y.inverse_transform(np.asarray(y_pred).reshape(-1, 1)).ravel()
        df_out['y_pred_orig'] = y_pred_orig
        if y_true is not None:
            y_true_orig = scaler_y.inverse_transform(np.asarray(y_true).reshape(-1, 1)).ravel()
            df_out['y_true_orig'] = y_true_orig
            print('Test SMAPE (orig):', smape_np(y_true_orig, y_pred_orig))
    df_out.to_csv('data_hh/result/pred_xgb_bayes.csv', index=False)
elif MODE == 'manual':
    tuned = None
    if 'USE_SAVED_BEST' in globals() and USE_SAVED_BEST and os.path.exists(BEST_JSON_PATH):
        tuned = load_best_params(BEST_JSON_PATH)
    elif MANUAL_PARAMS:
        tuned = dict(MANUAL_PARAMS)
    else:
        raise ValueError('manual 模式需要：USE_SAVED_BEST=True 且 JSON 存在，或在 MANUAL_PARAMS 中填写参数。')
    num_boost_round = int(tuned.pop('num_boost_round')) if 'num_boost_round' in tuned else 500
    params = dict(BASE_PARAMS)
    if 'lambda_' in tuned and 'lambda' not in tuned:
        tuned['lambda'] = float(tuned.pop('lambda_'))
    params.update(tuned)
    model = xgb.train(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        evals=evals,
        early_stopping_rounds=10,
        feval=smape_eval,
        maximize=False,
        verbose_eval=10,
    )
    y_pred = model.predict(dtest)
    try:
        y_true = y_test.values if hasattr(y_test, 'values') else y_test
    except NameError:
        y_true = None
    df_out = pd.DataFrame({'y_pred': y_pred})
    if y_true is not None:
        df_out['y_true'] = np.asarray(y_true).reshape(-1)
        print('Test SMAPE:', smape_np(df_out['y_true'], df_out['y_pred']))
    # 反标准化并计算原始尺度 SMAPE（若提供 scaler_y）
    try:
        scaler_y
        has_scaler = True
    except NameError:
        has_scaler = False
    if has_scaler and hasattr(scaler_y, 'inverse_transform'):
        y_pred_orig = scaler_y.inverse_transform(np.asarray(y_pred).reshape(-1, 1)).ravel()
        df_out['y_pred_orig'] = y_pred_orig
        if y_true is not None:
            y_true_orig = scaler_y.inverse_transform(np.asarray(y_true).reshape(-1, 1)).ravel()
            df_out['y_true_orig'] = y_true_orig
            print('Test SMAPE (orig):', smape_np(y_true_orig, y_pred_orig))
    model.save_model('data_hh/result/model/xgb_model_manual.json')
    df_out.to_csv('data_hh/result/pred_xgb_manual.csv', index=False)
else:
    raise ValueError("MODE 仅支持 'bayes' 或 'manual'")


/data/zy/anaconda3/envs/machine/lib/python3.10/site-packages/xgboost/training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


[0]	train-rmse:0.89414	train-SMAPE:158.54132	validation-rmse:0.89223	validation-SMAPE:158.65868
[10]	train-rmse:0.46423	train-SMAPE:74.80370	validation-rmse:0.46927	validation-SMAPE:75.73622
[20]	train-rmse:0.40065	train-SMAPE:65.48978	validation-rmse:0.41170	validation-SMAPE:66.80798
[30]	train-rmse:0.37680	train-SMAPE:62.07547	validation-rmse:0.39321	validation-SMAPE:63.93119
[40]	train-rmse:0.36457	train-SMAPE:60.45103	validation-rmse:0.38482	validation-SMAPE:62.72417
[50]	train-rmse:0.35757	train-SMAPE:59.51992	validation-rmse:0.38049	validation-SMAPE:62.07197
[60]	train-rmse:0.34991	train-SMAPE:58.50658	validation-rmse:0.37579	validation-SMAPE:61.38211
[70]	train-rmse:0.34280	train-SMAPE:57.58715	validation-rmse:0.37143	validation-SMAPE:60.76738
[80]	train-rmse:0.33654	train-SMAPE:56.76965	validation-rmse:0.36793	validation-SMAPE:60.24028
[90]	train-rmse:0.33248	train-SMAPE:56.24777	validation-rmse:0.36582	validation-SMAPE:59.92695
[100]	train-rmse:0.32789	train-SMAPE:55.65770	val